## Face Unlocker 


### Step 1 - Data Generating be ready infront of camera

In [ ]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(1)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

### Step 2 - Train Model

In [ ]:
cap.release()

In [1]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = './faces/user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

vimal_model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
vimal_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


Model trained sucessefully


### Step 3 - Be ready infront of camera it will recognize face and send you whatsapp message, mail and create ec2 instance and volume, attach with instance

In [2]:
import cv2
import numpy as np
import subprocess as sb
import os
import pywhatkit as kit

#Some Importand thing for WhatsApp
time =sb.getoutput("time/t")
hrs  = time[:2]
minute = time[3:5]
hrs =int(hrs)
minute = int(minute) + 1
mode =time[6:]
if mode=='PM':
    hrs = hrs + 12

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(1)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = vimal_model.predict(face)
        # harry_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 80:
            cv2.putText(image, "Hey Vimal", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            # Sending WhatsApp Message
            kit.sendwhatmsg("+918083455620" , "hello" ,hrs , minute,wait_time=20)
            print("Wooho!! I send a alert message... on whatsapp")
            # Sends you  a mail after recognition
            #------------------ For mail
            import smtplib
            import imghdr
            from email.message import EmailMessage

            Sender_Email = "mdcba3@gmail.com"
            Reciever_Email = "sgn9556@gmail.com"
            Password = "AWSdada@786"#input('Enter your email account password: ')

            newMessage = EmailMessage()                         
            newMessage['Subject'] = "Yes I recognize your face" 
            newMessage['From'] = Sender_Email                   
            newMessage['To'] = Reciever_Email                   
            newMessage.set_content('Thanks For choosing!!') 


            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:

                smtp.login(Sender_Email, Password)              
                smtp.send_message(newMessage)
            print("Wooho It Sends the Mail ...")
            #Now Launching Ec2 Instances.. Over EC2 Instance
            os.system("aws ec2 run-instances  --image-id ami-011c99152163a87ae --instance-type t2.micro  --count 1")
            os.system("sleep 30")
            print("It sucessfully Launch the Instance over the AWS..")
            #Now creating the volume size 5GB
            os.system("aws ec2 create-volume --size 5  --availability-zone ap-south-1a")
            #Now retriving the Id of Instance and Volume
            volume_id = os.popen('aws ec2 describe-volumes --filters  Name=availability-zone,Values=ap-south-1a Name=status,Values=available --query Volumes[*].VolumeId --output text').read()
            final_volume_id = volume_id[0:len(volume_id)-1]
            print("The volume id of 5 GB EBS volume ",final_volume_id)
            instance_id = os.popen('aws ec2 describe-instances --filters Name=instance-state-name,Values=running --query "Reservations[].Instances[].InstanceId" --output text').read()
            final_instance_id = instance_id[0:len(instance_id)-1]
            print("The current running instance id ", final_instance_id)
            #Now attaching the Volume to Instance
            os.system('aws ec2 attach-volume --volume-id {0} --instance-id {1} --device /dev/sdf'.format(final_volume_id, final_instance_id))
            
            
            break
         
        else:
            
            cv2.putText(image, "I dont know, how r u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()     

<>:24: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:24: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-2-a294833c40a8>:24: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In 1 seconds web.whatsapp.com will open and after 20 seconds message will be delivered
Wooho!! I send a alert message... on whatsapp
Wooho It Sends the Mail ...
It sucessfully Launch the Instance over the AWS..
The volume id of 5 GB EBS volume  vol-042026fdba43832fe
The current running instance id  i-0f2b23e57aab826da
